In [ ]:
!pip install transformers
!pip install datasets
!pip install shap

In [ ]:
import shap
from transformers import AutoTokenizer
import numpy as np
import os
from transformers import Trainer, TrainingArguments,AutoModelForCausalLM, OPTForSequenceClassification, set_seed
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 40.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.9 MB/s eta 0:00:0

In [ ]:
!pip install datasets
from datasets import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
seed = 42
set_seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

no_deprecation_warning=True

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd  "drive/MyDrive/TUM/TUM XAI/classifiers/Galactica"
# %cd  "drive/MyDrive/artific/Classifiers/Galactica"

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1m6AYZbd3XOUMFqsgOLrRp8rl9q4ZPFfZ/TUM XAI/classifiers/Galactica


In [5]:
!ls

INCOMPLETEHuggingFaceIntegration.ipynb	galactica_classification.ipynb
README.md				galactica_complete_preds.csv
explainability.ipynb			results


In [4]:
!ls ../data

 archive				 data_nlp2022
 classifier_input_restricted.csv	 realnlp2022_restricted_4000.csv
 classifier_input_restricted_test.csv	'stats and cleaning.ipynb'
 classifier_input_restricted_train.csv


In [3]:
DATA_PATH = os.path.join("..", "data")
test_df = pd.read_csv(os.path.join(DATA_PATH, "classifier_input_restricted_test.csv"))
gpt3_ood_test_df = pd.read_csv(os.path.join(DATA_PATH, "data_nlp2022", "gpt3curienlp2022_restricted_test.csv"))
galactica_preds_full = pd.read_csv(os.path.join("galactica_complete_preds.csv")) # has predictions and labels

NameError: name 'os' is not defined

In [ ]:
galactica_preds_full

,Unnamed: 0,text,label,src,prediction,0,1
0,0,Abstract:\n\n The construction of XML is an ap...,1,scigen,1,4.927146e-07,1.000000
1,1,Abstract:\n\n Semaphores and ﬁber-optic cables...,1,scigen,1,4.154256e-07,1.000000
2,2,Abstract:\n\nThe general properties of gravita...,1,galactica,1,7.248534e-07,0.999999
3,3,Abstract:\n\nRecent calculations have pointed ...,0,real,0,9.999114e-01,0.000089
4,4,Abstract:\n\nRecent years have witnessed a rem...,0,real,0,9.999965e-01,0.000003
...,...,...,...,...,...,...,...
7995,7995,Abstract:\n\nWe present the results of a syste...,1,gpt2,1,1.250637e-06,0.999999
7996,7996,Abstract:\n\n Mathematicians agree that knowle...,1,scigen,1,4.013844e-07,1.000000
7997,7997,"Abstract:\n\nIn this paper, we study the effec...",1,gpt2,1,3.809361e-07,1.000000
7998,7998,Abstract:\n\nNear Field Communication (NFC) st...,0,real,0,9.999716e-01,0.000028


In [ ]:
model_path = os.path.join(".", "results", "82-trainedOnNewData")

In [ ]:
model = OPTForSequenceClassification.from_pretrained(model_path)
model = model.to("cuda")
model.eval()
model.config.is_decoder=True

In [ ]:
model_params_size = "125m"
tokenizer = AutoTokenizer.from_pretrained(f"facebook/galactica-{model_params_size}", pad_token = "<pad>", eos_token = "</s>")
# restrict to only 2048 tokens input.
tokenizer.model_max_length = 2048

tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id

# # pad left since decoder only architecture and the last token is used for classification, unlike bert.
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
tokenizer

PreTrainedTokenizerFast(name_or_path='facebook/galactica-125m', vocab_size=50000, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='left', special_tokens={'eos_token': '</s>', 'pad_token': '</s>'})

In [ ]:
def transform_df(df, start_of_text = None, end_of_text = None) -> pd.DataFrame:
  """
    Transforms a pandas DataFrame by concatenating the "abstract", "introduction", and "conclusion" columns into a new "text" column, with optional separators and text to prepend and append to the "text" column.

    Args:
        df (pandas.DataFrame): The input DataFrame to transform.
        sep (str, optional): A string to separate the text from the label in the output DataFrame. Defaults to None.
        start_of_text (str, optional): A string to prepend to the beginning of the text. Defaults to None.
        end_of_text (str, optional): A string to append to the end of the text. Defaults to None.

    Returns:
        pandas.DataFrame: The transformed DataFrame with a "text" column containing the concatenated text, and optionally a "label" and "src" column.
  """
  end_of_text = end_of_text if end_of_text else ""
  start_of_text = start_of_text if start_of_text else ""
  # Create text
  df["text"] = start_of_text \
    + "Abstract:\n\n" + df["abstract"] \
    + "\n\nIntroduction:\n\n" + df["introduction"] \
    + "\n\nConclusion:\n\n" + df["conclusion"] \
    + end_of_text

  return df[["text", "label", "src"]]

In [ ]:
def pred(x):
  # move input to array
  new = []
  for i in range(len(x)):
    new.append(x[i])
  x = new
  inputs = tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=1024).to("cuda")
  inputs.pop("token_type_ids")
  outputs = model(**inputs)
  probs = outputs.logits.softmax(-1)
  return probs.to('cpu').detach().numpy()

In [ ]:
import matplotlib.pyplot as plt
def explain(df, n = 5):
  random_preds = df.sample(n=n, random_state = 42)
  display(random_preds)
  df = random_preds["text"].tolist()
  explainer = shap.Explainer(pred, tokenizer)
  shap_values = explainer(df)
  shap.plots.text(shap_values[:,:,0])
  shap.plots.bar(shap_values[:,:,0].mean(0)) 

In [ ]:
transformed_test = transform_df(test_df)
transformed_gpt3 = transform_df(gpt3_ood_test_df)

In [ ]:
explain(transformed_gpt3)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
explain(transformed_test[transformed_test["src"] != "real"])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
explain(transformed_test[transformed_test["src"] == "real"])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
explain(galactica_preds_full[galactica_preds_full["label"] != galactica_preds_full["prediction"]])

Output hidden; open in https://colab.research.google.com to view.